<center><h1><b>NYC Apartment Search</b></h1></center>

---
# Setup

## Import Statements

This code block includes various import statements that bring in libraries and modules necessary for data manipulation, visualization, database operations, and geographic information system (GIS) tasks. These imports cover a wide range of functionalities required for the project.



In [2]:
# Standard library imports
import os
import json
import unittest
import pathlib
import urllib.parse
from datetime import datetime
from pathlib import Path
from typing import List

# Data manipulation and visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import requests
import logging

# Database and GIS related imports
import psycopg2
import psycopg2.extras
import geoalchemy2 as gdb
import geopandas as gpd
import shapely
from shapely.geometry import Point, Polygon, mapping
from shapely import wkb

# SQLAlchemy imports
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import (
    create_engine,
    Column,
    Integer,
    Float,
    String,
    DateTime,
    text,
    ForeignKey,
)
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.schema import CreateTable
from geoalchemy2 import Geometry
from geoalchemy2.shape import to_shape


## Constants and File Paths

This code block defines several constants and file paths for data directories and files related to the project. It also includes constants for accessing New York City data through an API.


In [3]:
# File paths and directory constants
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

# New York City data API constants
NYC_DATA_APP_TOKEN = "SMg9akfNT3gV1L4QAEb8vlx4F"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

In [4]:
# Database constants
DB_NAME = "group19project"
DB_USER = "cecilialin"
DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"

In [5]:
# Directory for DB queries for Part 3
QUERY_DIR = pathlib.Path("queries")

# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

In [6]:
# Directory path
directory = "data/resource"

# Create the directory. exist_ok=True means it won't throw an error if the directory already exists.
os.makedirs(directory, exist_ok=True)

print(f"Directory {directory} has been created.")


Directory data/resource has been created.


## Logging Setup

This code block sets up logging for information (INFO) and error tracking. It configures the logging module to handle log messages for the current module.


In [7]:
# Setup logging for info and error tracking
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

---
# Part 1: Data Preprocessing

In Part 1 of the project, the primary focus is on data preprocessing. This stage involves downloading datasets, both manually and programmatically, cleaning and filtering the data, filling in missing values, and generating relevant data samples. 

## New York City GeoJSON Data Downloading

This function downloads New York City GeoJSON data in batches and saves them to individual files. Key features include:

- **Batch Downloading**: Downloads data in manageable batches, controlled by the `limit` parameter.
- **File Saving**: Each batch is saved as a separate GeoJSON file for efficient data handling.
- **Error Handling and Logging**: Incorporates error handling for robust data retrieval and logs progress for tracking.

In [8]:
def download_nyc_geojson_data(url: str, limit: int = 1000000) -> List[pathlib.Path]:
    """
    Downloads NYC GeoJSON data in batches and saves each batch to a separate file.

    :param url: URL to download the data from.
    :param limit: Number of records per batch to download (default is 1,000,000).
    :return: List of file paths of the downloaded GeoJSON files.
    """
    
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    total_record_count = 0
    file_index = 0
    more_data_available = True

    while more_data_available:
        current_filename = DATA_DIR / f"{url_path}_{file_index}.geojson"
        if not current_filename.exists():
            logger.info(f"Downloading data to {current_filename}...")
            try:
                params = {'$limit': limit, '$offset': file_index * limit}
                response = requests.get(url, params=params)
                response.raise_for_status()
                data = response.json()
                if data:
                    with open(current_filename, "w") as f:
                        json.dump(data, f)
                    total_record_count += len(data)

                    # Check if the returned data is less than the requested limit
                    if len(data) < limit:
                        more_data_available = False
                    else:
                        file_index += 1

                else:
                    more_data_available = False
            except requests.RequestException as e:
                logger.error(f"Failed to retrieve data: {e}")
                break
        else:
            logger.info(f"File {current_filename} already exists. Skipping download.")
            file_index += 1

    logger.info(f"Total records downloaded: {total_record_count}")
    return [DATA_DIR / f"{url_path}_{i}.geojson" for i in range(file_index)]
